## Landsat Phenology with Cloud Filtering

In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import rioxarray as rio
import xrspatial.multispectral as ms

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
import odc
from odc.stac import stac_load
from odc.algo import to_rgba

In [2]:
#read_csv
df = pd.read_csv('Crop_Location_Data_20221201.csv')
df.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [158]:
lat_longs = list(df['id'].apply(lambda x: x[1:-1]))
box_size_deg = 0.05
time_window="2021-11-01/2022-08-31"
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")


In [159]:
# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees

resolution =  10 # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 

In [160]:
# To mask the pixels and find clouds or water, it is best to use the bit values of the 16-bit qa_pixel flag
# See the website above for a nice explanation of the process

bit_flags = {
            'fill': 1<<0,
            'dilated_cloud': 1<<1,
            'cirrus': 1<<2, 
            'cloud': 1<<3,
            'shadow': 1<<4, 
            'snow': 1<<5, 
            'clear': 1<<6,
            'water': 1<<7
}

In [161]:
# Create a function that will mask pixels with a given type

def get_mask(mask, flags_list):
    
    # Create the result mask filled with zeros and the same shape as the mask
    final_mask = np.zeros_like(mask)
    
    # Loop through the flags  
    for flag in flags_list:
        
        # get the mask for each flag
        flag_mask = np.bitwise_and(mask, bit_flags[flag])
        
        # add it to the final flag
        final_mask = final_mask | flag_mask
    
    return final_mask > 0

In [174]:
#define column
Latitude_Longtitudes = list()
time_frames = list()
original_red = list()
original_blue= list()
original_green = list()
original_ndvi = list()
filter_red = list()
filter_blue= list()
filter_green = list()
filter_ndvi = list()

filter_lir = list()
filter_swir= list()
original_lir= list()
original_swir = list()

In [189]:
for lat_long in lat_longs:
    lat_long = tuple(map(float, lat_long.split(', ')))
    min_lon = lat_long[1]-box_size_deg/2
    min_lat = lat_long[0]-box_size_deg/2
    max_lon = lat_long[1]+box_size_deg/2
    max_lat = lat_long[0]+box_size_deg/2
    bounds = (min_lon, min_lat, max_lon, max_lat)
    
    search = stac.search(
    collections=["landsat-c2-l2"],  #Landsat Collection 2 Level-2  #https://planetarycomputer.microsoft.com/dataset/group/landsat
    bbox=bounds, 
    datetime=time_window,
    query={"platform": {"in": ["landsat-8", "landsat-9"]},},
    )
    
    items = search.get_all_items()
    
    # https://odc-stac.readthedocs.io/en/latest/notebooks/stac-load-S2-ms.html#Configuration
    xx = stac_load(
    items,
    #bands=["red", "green", "blue", "nir08", "qa_pixel"],  
    bands=["red", "green", "blue", "nir08", "qa_pixel", "swir16", "lwir11"],  
            #bands to calculate nvdi: "red", "nir08"
            #bands for adjust brightness of images: "red", "green", "blue"
            #bands for filtering mask:  "qa_pixel", "qa_radsat", "qa_aerosol"
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    patch_url=pc.sign,
    bbox=bounds
    )
    
    # Apply scaling and offsets for Landsat Collection-2 (reference below) to the spectral bands ONLY
    # https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2
    xx['red'] = (xx['red']*0.0000275)-0.2
    xx['green'] = (xx['green']*0.0000275)-0.2
    xx['blue'] = (xx['blue']*0.0000275)-0.2
    xx['nir08'] = (xx['nir08']*0.0000275)-0.2
    
    if (len(xx.time) == 59) or (len(xx.time) == 58) or (len(xx.time) == 57):
        xx = xx.drop_isel(time=[len(xx.time) - 10])
    if len(xx.time) == 31:
        xx = xx.drop_isel(time=[25])
    if (len(xx.time) == 63) or (len(xx.time) == 64) or (len(xx.time) == 65):
        xx = xx.drop_isel(time=[50,51,52])
    if len(xx.time) == 91:
        xx = xx.drop_isel(time=[74,75])
    if (len(xx.time) == 66) or (len(xx.time) == 67):
        xx = xx.drop_isel(time=[53,54])
        
    full_mask = get_mask(xx['qa_pixel'], ['fill', 'dilated_cloud', 'cirrus', 'cloud', 'shadow', 'water'])
    cleaned_data = xx.where(~full_mask)
    
    # Calculate the mean of the data across the sample region and then NDVI
    # Perform this calculation for the unfiltered and cloud-filtered (clean) datasets
    time_frame = xx.time.values
    
    mean_unfiltered = xx.mean(dim=['longitude','latitude']).compute()
    mean_red = mean_unfiltered.red.values
    mean_blue = mean_unfiltered.blue.values
    mean_green = mean_unfiltered.green.values
    ndvi_mean = ((mean_unfiltered.nir08-mean_unfiltered.red)/(mean_unfiltered.nir08+mean_unfiltered.red)).values
    mean_swir = mean_unfiltered.swir16.values
    mean_lwir = mean_unfiltered.lwir11.values

    mean_clean = cleaned_data.mean(dim=['longitude','latitude']).compute()
    mean_red_clean = mean_clean.red.values
    mean_blue_clean = mean_clean.blue.values
    mean_green_clean = mean_clean.green.values
    ndvi_mean_clean = ((mean_clean.nir08-mean_clean.red)/(mean_clean.nir08+mean_clean.red)).values
    mean_swir_clean = mean_clean.swir16.values
    mean_lwir_clean = mean_clean.lwir11.values
    
    Latitude_Longtitudes.extend([lat_long]*len(time_frame))
    time_frames.extend(time_frame)
    original_red.extend(mean_red)
    original_blue.extend(mean_blue)
    original_green.extend(mean_green)
    original_ndvi.extend(ndvi_mean)
    original_lir.extend(mean_lwir)
    original_swir.extend(mean_swir)
    
    
    filter_red.extend(mean_red_clean)
    filter_blue.extend(mean_blue_clean)
    filter_green.extend(mean_green_clean)
    filter_ndvi.extend(ndvi_mean_clean)
    filter_lir.extend(mean_lwir_clean)
    filter_swir.extend(mean_swir_clean)

In [191]:
df_new = pd.DataFrame({'Latitude_Longtitudes': Latitude_Longtitudes,
        'time_frames': time_frames,
        'original_red':original_red ,
        'original_blue':original_blue,
        'original_green': original_green,
        'original_ndvi' :original_ndvi,
        'filter_red':filter_red,
        'filter_blue':filter_blue,
        'filter_green':filter_green ,
        'filter_ndvi': filter_ndvi,
        'filter_lir': filter_lir,
        'original_lir': original_lir,
        'original_swir': original_swir,
        'filter_swir': filter_swir})

In [193]:
df = df_new.merge(df, left_on='Latitude and Longitude', right_on='Latitude_Longtitudes', how='inner')

In [194]:
df.to_csv('train-Landsat.csv')

In [195]:
df = pd.read_csv('

,Latitude_Longtitudes,time_frames,original_red,original_blue,original_green,original_ndvi,filter_red,filter_blue,filter_green,filter_ndvi,filter_lir,original_lir,original_swir,filter_swir
0,"(10.18019073690894, 105.32022315786804)",2021-11-05 03:14:47.989597,0.142774,0.058752,0.131125,0.070088,0.103936,0.005552,0.090220,0.159310,45976.325313,45390.355224,11047.001185,9602.867561
1,"(10.18019073690894, 105.32022315786804)",2021-11-10 03:17:16.550936,0.508067,0.522658,0.509350,0.029567,NaN,NaN,NaN,NaN,NaN,464.873402,21099.404600,NaN
2,"(10.18019073690894, 105.32022315786804)",2021-11-15 03:19:44.586755,0.653280,0.678538,0.658584,0.006147,NaN,NaN,NaN,NaN,NaN,35355.275841,23568.800076,NaN
3,"(10.18019073690894, 105.32022315786804)",2021-11-15 03:20:46.455385,0.564710,0.587885,0.567710,0.008691,NaN,NaN,NaN,NaN,NaN,31536.195190,21231.673739,NaN
4,"(10.18019073690894, 105.32022315786804)",2021-11-24 03:14:35.582976,0.164490,0.092888,0.152713,0.133362,NaN,NaN,NaN,NaN,NaN,31247.535711,11321.943359,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12721,"(10.574733898351617, 105.10410108072531)",2022-07-29 03:20:25.214161,-0.053808,-0.070183,-0.054404,-2.116853,0.081451,0.045770,0.081587,0.598850,46194.635733,17636.863928,6220.227412,13888.078187
12722,"(10.574733898351617, 105.10410108072531)",2022-07-29 03:20:49.117908,0.160267,0.119092,0.157827,0.365255,0.079956,0.043208,0.077249,0.554973,46532.350254,45505.497761,15623.869458,13331.128344
12723,"(10.574733898351617, 105.10410108072531)",2022-08-06 03:20:08.865204,0.257327,0.287186,0.266147,-0.013680,NaN,NaN,NaN,NaN,NaN,127.171433,6199.467899,NaN
12724,"(10.574733898351617, 105.10410108072531)",2022-08-06 03:20:32.777400,0.854660,0.924958,0.875648,-0.013508,NaN,NaN,NaN,NaN,NaN,293.000000,14388.312348,NaN
